In [17]:
import pandas as pd
import requests
import urllib.request
import re
import time
from bs4 import BeautifulSoup

In [18]:
def next_url(soup):
    link=soup.find(class_='lister-page-next next-page')
    if link is None:
        return None
    else:
        link=link.get('href')
        link='https://www.imdb.com'+link
    return link

In [19]:
def url_get(url=''):
    if url=='':
        url='https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&ref_=adv_prv'
    user_agent={}
    user_agent['Mozila'] = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
    user_agent['Android'] = 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:68.0) Gecko/20100101 Firefox/68.0'
    headers={'User-Agent':user_agent['Android'],'Accept-Language': 'en-US,en;q=0.8'}
    urlHandle=urllib.request.Request(url,None,headers)
    urlHandle=urllib.request.urlopen(urlHandle)
    content=urlHandle.read()
    return BeautifulSoup(content,'html.parser')

In [20]:
def DataFrame():
    data={}
    data['Name']=name
    data['Year of Release']=year
    data['Genre']=genre
    data['IMDB Rating']=IMDB_rating
    data['Metascore']=metascore
    data['Director(s)']=directors
    data['Star(s)']=stars
    data['Summary']=summary
    data['Votes']=votes
    data['Gross Earning']=grosses
    df = pd.DataFrame(data=data,index=index)
    df.to_csv('IMDB.csv')
    return df

In [21]:
def index_extract(movie):
    index_elem=movie.find(class_='lister-item-index unbold text-primary')
    
    if index_elem is None:
        return None
    else:
        ans=index_elem.text
        return int(ans.split('.')[0])

In [22]:
def name_extract(movie):
    header_elem=movie.find(class_='lister-item-header')
    if header_elem is None:
        return None
    else:
        header=header_elem.find('a').text
        return header

In [23]:
def year_extract(movie):
    year_elem=movie.find(class_='lister-item-year text-muted unbold')
    ans=re.findall(r'\((\d+)\)',year_elem.text)
    if len(ans)<1:
        return None
    else:
        return int(ans[0])

In [24]:
def genre_extract(movie):
    ans=movie.find(class_='genre')
    if ans is None:
        return None
    else:
        return ans.text.strip().replace('\n','')

In [25]:
def imdb_extract(movie):
    rating_elem=movie.find(class_='inline-block ratings-imdb-rating')
    if rating_elem is None:
        return None
    else:
        rating=rating_elem.find('strong').text
        return float(rating)

In [26]:
def metascore_extract(movie):
    temp=movie.find(class_='metascore')
    if temp is None:
        return None
    else:
        return str(temp.text.strip())

In [27]:
def summary_extract(movie):
    temp=movie.find_all('p',{'class':'text-muted'})
    if temp is None:
        return None
    else:
        temp=temp[-1].text
        return str(temp.replace('\n','').strip())

In [28]:
def votes_extract(movie):
    temp=movie.find(class_='sort-num_votes-visible')
    temp=temp.text.strip().replace('\n','')
    vote=re.findall(r'Vote[s:]+([0-9,]+)',temp)
    if len(vote)<1:
        return None
    else:
        return vote[0]

In [29]:
def gross_extract(movie):
    temp=movie.find(class_='sort-num_votes-visible')
    temp=temp.text.strip().replace('\n','')
    gross=re.findall(r'Gross:([\$0-9.,A-Z]+)',temp)
    if len(gross)<1:
        return None
    else:
        return gross[0]

In [30]:
def director_star_extract(movie):
    send=[]
    temp=''
    director_star_elem=movie.find_all('p',{'class':''})
    for i in director_star_elem:
        if 'Director' in i.text:
            temp=i.text
    temp=temp.replace('\n','').strip()
    director=re.findall(r'^Director[s:]*([a-zA-Z0-9 ,.]*)',temp)
    if len(director)<1:
        send.append(None)
    else:
        send.append(director[0])
    star=re.findall(r'Star[s:]+([a-zA-Z0-9 ,.]*)',temp)
    if len(star)<1:
        send.append(None)
    else:
        send.append(star[0])
    return send

In [31]:
index=[]
name=[]
year=[]
genre=[]
IMDB_rating=[]
metascore=[]
summary=[]
directors=[]
stars=[]
votes=[]
grosses=[]

def main():
    wait=input('Enter number of seconds to wait before fetching the next page (default: 10): ')
    if wait=='':
        wait=25
    else:
        wait=int(wait)
    print('Fetching page')
    soup=url_get()
    for i in range(1,6):
        all_movies=soup.find_all(class_='lister-item-content')
        print('Fetching page')
        print('Scraping.....')
        for movie in all_movies:
            index.append(index_extract(movie))
            name.append(name_extract(movie))
            year.append(year_extract(movie))
            genre.append(genre_extract(movie))
            IMDB_rating.append(imdb_extract(movie))
            metascore.append(metascore_extract(movie))
            summary.append(summary_extract(movie))
            director_and_star=director_star_extract(movie)
            directors.append(director_and_star[0])
            stars.append(director_and_star[1])
            votes.append(votes_extract(movie))
            grosses.append(gross_extract(movie))
        next_link=next_url(soup)
        if next_link is None:
            print('Scraping completed..Exiting')
            break;
        print('Scraping completed... sleeping for ',wait,' seconds')
        time.sleep(wait)
        soup=url_get(next_link)
    df=DataFrame()
    return df

In [32]:
df=main()

Enter number of seconds to wait before fetching the next page (default: 10): 10
Fetching page
Fetching page
Scraping.....
Scraping completed... sleeping for  10  seconds
Fetching page
Scraping.....
Scraping completed... sleeping for  10  seconds
Fetching page
Scraping.....
Scraping completed... sleeping for  10  seconds
Fetching page
Scraping.....
Scraping completed... sleeping for  10  seconds
Fetching page
Scraping.....
Scraping completed..Exiting


In [33]:
df

,Name,Year of Release,Genre,IMDB Rating,Metascore,Director(s),Star(s),Summary,Votes,Gross Earning
1,The Shawshank Redemption,1994,Drama,9.3,80,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,"2,124,308",$28.34M
2,The Godfather,1972,"Crime, Drama",9.2,100,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke...",The aging patriarch of an organized crime dyna...,"1,458,558",$134.97M
3,The Dark Knight,2008,"Action, Crime, Drama",9.0,84,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as The Joker emerges fro...,"2,089,372",$534.86M
4,The Godfather: Part II,1974,"Crime, Drama",9.0,90,Francis Ford Coppola,"Al Pacino, Robert De Niro, Robert Duvall, Dian...",The early life and career of Vito Corleone in ...,"1,014,426",$57.30M
5,The Lord of the Rings: The Return of the King,2003,"Adventure, Drama, Fantasy",8.9,94,Peter Jackson,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",Gandalf and Aragorn lead the World of Men agai...,"1,511,434",$377.85M
6,Pulp Fiction,1994,"Crime, Drama",8.9,94,Quentin Tarantino,"John Travolta, Uma Thurman, Samuel L. Jackson,...","The lives of two mob hitmen, a boxer, a gangst...","1,664,203",$107.93M
7,Schindler's List,1993,"Biography, Drama, History",8.9,93,Steven Spielberg,"Liam Neeson, Ralph Fiennes, Ben Kingsley, Caro...","In German-occupied Poland during World War II,...","1,102,320",$96.90M
8,12 Angry Men,1957,Drama,8.9,96,Sidney Lumet,"Henry Fonda, Lee J. Cobb, Martin Balsam, John ...",A jury holdout attempts to prevent a miscarria...,"605,022",$4.36M
9,Inception,2010,"Action, Adventure, Sci-Fi",8.8,74,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon",A thief who steals corporate secrets through t...,"1,862,512",$292.58M
10,Fight Club,1999,Drama,8.8,66,David Fincher,"Brad Pitt, Edward Norton, Meat Loaf, Zach Grenier",An insomniac office worker and a devil-may-car...,"1,697,974",$37.03M
